# Qwen-VL-Chat Test (4B params)

Chat-optimized VLM with bilingual support and good performance.

**Model:** `Qwen/Qwen-VL-Chat`  
**Size:** 4B parameters  
**License:** Permissive  
**Features:** Chat-optimized, bilingual (English/Chinese), zero-shot object detection  
**Requirements:** ~8GB disk, ~4GB RAM/VRAM


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image
from vlm_utils import get_device_info, load_test_images, display_image, print_section, print_subsection

device = get_device_info()


Using device: mps
PyTorch version: 2.9.1
Using Apple Silicon MPS (Metal Performance Shaders)


## Load Test Images


In [2]:
image_files = load_test_images()


Found 1 image(s) to test:
  - sample_image.jpg


## Load Qwen-VL-Chat Model


In [3]:
print("Loading Qwen-VL-Chat...")
model_id = "Qwen/Qwen-VL-Chat"

# Determine dtype based on device
# Note: Qwen-VL-Chat works best on CPU or CUDA, MPS has some issues
if device.type == 'mps':
    print("⚠️  Using CPU for better stability (Qwen-VL has some MPS quirks)")
    actual_device = torch.device('cpu')
    model_dtype = torch.float32
else:
    actual_device = device
    use_float16 = torch.cuda.is_available()
    model_dtype = torch.float16 if use_float16 else torch.float32

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, revision="462bad4a9c44c1e216c6b29f9469e0563b4f413d")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    device_map=actual_device,
    dtype=model_dtype,
    revision="462bad4a9c44c1e216c6b29f9469e0563b4f413d"
).eval()
print("✓ Qwen-VL-Chat loaded!")


Loading Qwen-VL-Chat...
⚠️  Using CPU for better stability (Qwen-VL has some MPS quirks)


ValueError: Unrecognized model in Qwen/Qwen-VL-Chat. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: aimv2, aimv2_vision_model, albert, align, altclip, apertus, arcee, aria, aria_text, audio-spectrogram-transformer, autoformer, aya_vision, bamba, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, bitnet, blenderbot, blenderbot-small, blip, blip-2, blip_2_qformer, bloom, blt, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, cohere2, cohere2_vision, colpali, colqwen2, conditional_detr, convbert, convnext, convnextv2, cpmant, csm, ctrl, cvt, d_fine, dab-detr, dac, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deepseek_v2, deepseek_v3, deepseek_vl, deepseek_vl_hybrid, deformable_detr, deit, depth_anything, depth_pro, deta, d

## Define Inference Function


In [ ]:
def describe_image(image_path, prompt="Describe this image in detail."):
    """Generate description for an image using Qwen-VL-Chat."""
    # Qwen-VL-Chat uses a special format with <img>path</img> tags
    query = tokenizer.from_list_format([
        {'image': str(image_path)},
        {'text': prompt},
    ])
    response, history = model.chat(tokenizer, query=query, history=None)
    return response


## Test on All Images


In [ ]:
for image_path in image_files:
    print_section(f"Image: {image_path.name}")
    
    display_image(image_path)
    
    print_subsection("🔍 Qwen-VL-Chat Description:")
    try:
        desc = describe_image(image_path)
        print(desc)
    except Exception as e:
        print(f"Error: {e}")


## Custom Prompts

Try asking specific questions about an image.


In [ ]:
if image_files:
    test_image = image_files[0]
    
    custom_prompts = [
        "What objects can you see in this image?",
        "What colors are prominent in this image?",
        "What is the main subject of this image?"
    ]
    
    print_section(f"Custom Prompts - {test_image.name}")
    display_image(test_image)
    
    for prompt in custom_prompts:
        print_subsection(f"Q: {prompt}")
        answer = describe_image(test_image, prompt)
        print(f"A: {answer}")
